In [2]:
# ! pip install numpy pandas matplotlib

In [5]:
## all imports
from IPython.display import HTML
import numpy as np

import time
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# from secret import *

<h2>How to load a table</h2>
<ul>
<li>we use Pandas for this
<li>Pandas can do a lot more
<li>more about it later
</ul>

<h2>The MovieLens data</h2>

http://grouplens.org/datasets/movielens/

<h3>Read the user data</h3>

In [6]:
# pass in column names for each CSV
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

users = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.user', 
    sep='|', names=u_cols)

users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


Read the ratings
============

In [7]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.data', 
    sep='\t', names=r_cols)

ratings.head() 

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Now data about the movies
=========================

In [8]:
# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']

movies = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.item', 
    sep='|', names=m_cols, usecols=range(5))

movies.head()

,movie_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)


Get information about data
=======================

In [11]:
print(movies.dtypes)
print()
print(movies.describe())
# *** Why only those two columns? ***

movie_id                int64
title                  object
release_date           object
video_release_date    float64
imdb_url               object
dtype: object
()
          movie_id  video_release_date
count  1682.000000                 0.0
mean    841.500000                 NaN
std     485.695893                 NaN
min       1.000000                 NaN
25%     421.250000                 NaN
50%     841.500000                 NaN
75%    1261.750000                 NaN
max    1682.000000                 NaN


Selecting data
==============

* DataFrame => group of Series with shared index
* single DataFrame column => Series

In [10]:
print users.head()
print
print users['occupation'].head()
print
columns_you_want = ['occupation', 'sex'] 
print(users[columns_you_want].head())

   user_id  age sex  occupation zip_code
0        1   24   M  technician    85711
1        2   53   F       other    94043
2        3   23   M      writer    32067
3        4   24   M  technician    43537
4        5   33   F       other    15213

0    technician
1         other
2        writer
3    technician
4         other
Name: occupation, dtype: object

   occupation sex
0  technician   M
1       other   F
2      writer   M
3  technician   M
4       other   F


Filtering data
==============

Select users older than 25

In [13]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [16]:
users.iloc[3]

user_id                4
age                   24
sex                    M
occupation    technician
zip_code           43537
Name: 3, dtype: object

In [17]:
oldUsers = users[users.age > 25]
oldUsers.head()

,user_id,age,sex,occupation,zip_code
1,2,53,F,other,94043
4,5,33,F,other,15213
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
7,8,36,M,administrator,05201


Quiz:
=====

* show users aged 40 and male

* show the mean age of female programmers

In [13]:
# users aged 40 AND male
users[(users.age == 40) & (users.sex == 'M')].head(3)

,user_id,age,sex,occupation,zip_code
18,19,40,M,librarian,02138
82,83,40,M,other,44133
115,116,40,M,healthcare,97232


In [14]:
## users who are female and programmers
selected_users = users[(users.sex == 'F') & 
                       (users.occupation == 'programmer')]

## show statistic summary
print selected_users.describe()
print

## alternatives:
print "Mean age of selected_users is ", selected_users.age.mean()
print "Mean age of selected_users is ", selected_users['age'].mean()

          user_id        age
count    6.000000   6.000000
mean   411.166667  32.166667
std    149.987222   5.115336
min    292.000000  26.000000
25%    313.000000  28.250000
50%    378.000000  32.000000
75%    416.750000  36.500000
max    698.000000  38.000000

Mean age of selected_users is  32.1666666667
Mean age of selected_users is  32.1666666667


Split-apply-combine
===================

* splitting the data into groups based on some criteria
* applying a function to each group independently
* combining the results into a data structure

Split-apply-combine
===================

<img src=http://i.imgur.com/yjNkiwL.png></img>

Find Diligent Users
===================

* split data per user ID
* count ratings
* combine result

In [15]:
print ratings.head()
print

grouped_data = ratings.groupby('user_id')
print list(grouped_data)

   user_id  movie_id  rating  unix_timestamp
0      196       242       3       881250949
1      186       302       3       891717742
2       22       377       1       878887116
3      244        51       2       880606923
4      166       346       1       886397596

[(1,        user_id  movie_id  rating  unix_timestamp
202          1        61       4       878542420
305          1       189       3       888732928
333          1        33       4       878542699
334          1       160       4       875072547
478          1        20       4       887431883
639          1       202       5       875072442
687          1       171       5       889751711
820          1       265       4       878542441
933          1       155       2       878542201
972          1       117       3       874965739
1167         1        47       4       875072125
1299         1       222       4       878873388
1382         1       253       5       874965970
1440         1       113       5      

In [26]:
## split data
grouped_data = ratings['movie_id'].groupby(ratings['user_id'])
print(list(grouped_data))

[(1, 202       61
305      189
333       33
334      160
478       20
639      202
687      171
820      265
933      155
972      117
1167      47
1299     222
1382     253
1440     113
1617     227
1780      17
1989      90
2328      64
3049      92
3059     228
3171     266
3191     121
3234     114
3246     132
3248      74
3260     134
3358      98
3377     186
3431     221
3710      84
        ... 
74577    165
74847    116
75385    198
77073    124
77238     95
77630    217
78171     58
78695    142
78817    216
81463    126
81568     83
82444    231
82786    204
83307      3
84793    207
85377    244
85557     19
86585     29
87831     18
87967     59
88021     15
88259    111
88262     52
88893     88
89876     13
92049     28
92487    172
94019    122
96699    152
99073     94
Name: movie_id, Length: 272, dtype: int64), (2, 700      292
924      251
1052      50
3425     314
5063     297
5324     290
6310     312
7519     281
7973      13
8253     280
9201     303
9980     30

In [27]:
## count and combine
ratings_per_user = grouped_data.count()

ratings_per_user.head(5)

user_id
1    272
2     62
3     54
4     24
5    175
Name: movie_id, dtype: int64

Quiz
====

* get the average rating per movie
* advanced: get the movie titles with the highest average rating

In [16]:
## split data
grouped_data = ratings['rating'].groupby(ratings['movie_id'])
## average and combine
average_ratings = grouped_data.mean()
print "Average ratings:"
print average_ratings.head()
print

Average ratings:
movie_id
1    3.878319
2    3.206107
3    3.033333
4    3.550239
5    3.302326
Name: rating, dtype: float64



In [17]:
maximum_rating = average_ratings.max()
print "Maximum rating = ", maximum_rating

Maximum rating =  5.0


In [18]:
good_movie_ids = average_ratings[average_ratings == maximum_rating].index
print good_movie_ids

Int64Index([814, 1122, 1189, 1201, 1293, 1467, 1500, 1536, 1599, 1653], dtype='int64', name=u'movie_id')


In [34]:
print "Best movie titles"
print movies[movies.movie_id.isin(good_movie_ids)].title
print

Best movie titles
813                         Great Day in Harlem, A (1994)
1121                       They Made Me a Criminal (1939)
1188                                   Prefontaine (1997)
1200           Marlene Dietrich: Shadow and Light (1996) 
1292                                      Star Kid (1997)
1466                 Saint of Fort Washington, The (1993)
1499                            Santa with Muscles (1996)
1535                                 Aiqing wansui (1994)
1598                        Someone Else's America (1995)
1652    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object



<h2>Will you trust the above results if you have to pick a movie to watch?</h2/>

In [42]:
how_many_ratings = grouped_data.count()
#print how_many_ratings
print "Number of ratings per movie"
print how_many_ratings[average_ratings == maximum_rating]

Number of ratings per movie
movie_id
814     1
1122    1
1189    3
1201    1
1293    3
1467    2
1500    2
1536    1
1599    1
1653    1
Name: rating, dtype: int64


Passing a Function
==================


In [43]:
average_ratings = grouped_data.apply(lambda f: f.mean())
average_ratings.head()

movie_id
1    3.878319
2    3.206107
3    3.033333
4    3.550239
5    3.302326
Name: rating, dtype: float64

Quiz
====

* get the average rating per user
* advanced: list all occupations and if they are male or female dominant

In [49]:
# Same as this
# ratings.groupby(ratings['user_id'])['rating']
# OR
# ratings.groupby('user_id')['rating']

grouped_data = ratings['rating'].groupby(ratings['user_id'])
average_ratings = grouped_data.mean()
average_ratings.head()

user_id
1    3.610294
2    3.709677
3    2.796296
4    4.333333
5    2.874286
Name: rating, dtype: float64

In [22]:
grouped_data = users['sex'].groupby(users['occupation'])
print grouped_data
def male_female(f):
    if sum(f == 'M') > sum(f == 'F'):
        return 'Male'
    else:
        return 'Female'

male_dominant_occupations = grouped_data.apply(male_female)
#male_dominant_occupations = grouped_data.apply(lambda f: 
#                                               sum(f == 'M') > sum(f == 'F'))
print male_dominant_occupations

occupation
administrator      Male
artist             Male
doctor             Male
educator           Male
engineer           Male
entertainment      Male
executive          Male
healthcare       Female
homemaker        Female
lawyer             Male
librarian        Female
marketing          Male
none               Male
other              Male
programmer         Male
retired            Male
salesman           Male
scientist          Male
student            Male
technician         Male
writer             Male
Name: sex, dtype: object


In [54]:
grouped_data = users['sex'].groupby(users['occupation'])
def male_female(f):
    if sum(f == 'M') > sum(f == 'F'):
        return 'Male'
    else:
        return 'Female'

male_dominant_occupations = grouped_data.apply(male_female)
#male_dominant_occupations = grouped_data.apply(lambda f: 
#                                               sum(f == 'M') > sum(f == 'F'))
print male_dominant_occupations

occupation
administrator      Male
artist             Male
doctor             Male
educator           Male
engineer           Male
entertainment      Male
executive          Male
healthcare       Female
homemaker        Female
lawyer             Male
librarian        Female
marketing          Male
none               Male
other              Male
programmer         Male
retired            Male
salesman           Male
scientist          Male
student            Male
technician         Male
writer             Male
Name: sex, dtype: object


In [51]:
print 'number of male users: '
print sum(users['sex'] == 'M')

print 'number of female users: '
print sum(users['sex'] == 'F')

number of male users: 
670
number of female users: 
273
